In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Reshape, Flatten
from keras.regularizers import L1L2

In [ ]:
from keras_adversarial import AdversarialModel, simple_gan, gan_targets

In [27]:
(previsores_treinamento, _), (_, _) = mnist.load_data()
previsores_treinamento = previsores_treinamento.astype('float32') / 255

In [ ]:
gerador = Sequential()
gerador.add(Dense(units=500, input_dim=100, activation='relu',
                  kernel_regularizer=L1L2(1e-5, 1e-5)))
gerador.add(Dense(units=500, activation='relu',
                  kernel_regularizer=L1L2(1e-5, 1e-5))) 
gerador.add(Dense(units=784, activation='sigmoid',
                  kernel_regularizer=L1L2(1e-5, 1e-5)))
gerador.add(Reshape((28, 28)))

discriminador = Sequential()
discriminador.add(Flatten(input_shape=(28, 28)))
discriminador.add(Dense(units=500, activation='relu',
                        kernel_regularizer=L1L2(1e-5, 1e-5)))
discriminador.add(Dense(units=500, activation='relu',
                        kernel_regularizer=L1L2(1e-5, 1e-5)))
discriminador.add(Dense(units=1, activation='sigmoid',
                        kernel_regularizer=L1L2(1e-5, 1e-5)))

gan = simple_gan(gerador, discriminador, normal_latent_sampling((100,)))
modelo = AdversarialModel(base_model=gan,
                          player_params=[gerador.trainable_weights,
                                          discriminador.trainable_weights])
modelo.adversarial_compile(adversarial_optimizer=AdversarialOptimizerSimultaneous(),
                            player_optimizers=['adam', 'adam'],
                            loss='binary_crossentropy')

modelo.fit(x=previsores_treinamento, y=gan_targets(60000),
            epochs=100, batch_size=256)

In [ ]:
amostras = np.random.normal(size=(10, 100))
previsao = gerador.predict(amostras)

In [ ]:
for i in range(previsao.shape[0]):
    plt.imshow(previsao[i, :], cmap='gray')
    plt.show()